# 2021 폐플라스틱 이미지 객체 검출 예측 경진대회
- 팀: (10) 진짜이대로내요
- 구성원:
    - 윤준호 (팀장)
    - 최미래
- 제출물:
    - 소스코드 ipynb 파일 (Detectron2 사용)
    - 학습 가중치 pth 파일

## Setup

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/competition/

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

In [5]:
# Import and setup logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# Import common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import random
import copy

# Import Detectron2 utilities
from detectron2 import model_zoo
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader, build_detection_train_loader
from detectron2.data import build_detection_test_loader, build_detection_train_loader
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T
from detectron2.evaluation import COCOEvaluator

## Dataset

In [ ]:
# Check bbox mode == (x, y ,w, h)
img = cv2.imread('./contest_dataset/train/image/PE/PE_074_148.jpg')
with open('./contest_dataset/train/annotation/PE/PE_074_148.json') as f:
    label = json.load(f)
    for instance in label['annotations']:
        x, y, w, h = instance['bbox']
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255))
cv2_imshow(img)

In [7]:
# Set classes
classes = ['pet', 'ps', 'pp', 'pe']

# Exclude file due to prevent "polygon" error
excluded = ['PET_015_1358.json', 'PS_038_2948.json', 'PS_042_7482.json', 'PS_038_4646.json', # len(seg_bitmask) <= 4
            'PS_037_5996.json', 'PS_035_7052.json'] # len(seg_bitmask) % 2 != 0


def get_contest_dicts(data_dir):
    image_dir = os.path.join(data_dir, 'image')
    label_dir = os.path.join(data_dir, 'annotation')

    dataset_all_classes = []
    for obj_cls in sorted(os.listdir(label_dir)):
        dataset_one_class = []
        for file_name in tqdm(os.listdir(os.path.join(label_dir, obj_cls))):
            if file_name in excluded:
                continue
            
            json_file = os.path.join(os.path.join(label_dir, obj_cls), file_name)
            with open(json_file) as f:
                label = json.load(f)
            record = {}
            record['file_name'] = os.path.join(image_dir, obj_cls, label['images'][0]['file_name'])
            for value in ['height', 'width']:
                record[value] = label['images'][0][value]
            record['image_id'] = label['images'][0]['id']
            
            objs = []
            for instance in label['annotations']:
                obj = {
                    "bbox": instance['bbox'],
                    "bbox_mode": BoxMode.XYWH_ABS,
                    "segmentation": instance['segmentation'],
                    # "category_id": instance['category_id']
                    "category_id": instance['category_id'] - 1
                }
                objs.append(obj)

            record['annotations'] = objs
            dataset_one_class.append(record)
        dataset_all_classes.append(dataset_one_class)
    
    return dataset_all_classes

In [ ]:
# Dataset split
dataset_train = get_contest_dicts('./contest_dataset/train')
dataset_test = get_contest_dicts('./contest_dataset/test')

train_dataset = []
val_dataset = []
test_dataset = []

for cls in range(4):
    one_class = dataset_train[cls] + dataset_test[cls]
    random.shuffle(one_class)
    split_pt1 = int(len(one_class) * 0.7)
    split_pt2 = split_pt1 + int(len(one_class) * 0.15)
    train_dataset += one_class[:split_pt1]
    val_dataset += one_class[split_pt1:split_pt2]
    test_dataset += one_class[split_pt2:]

In [9]:
# Register dataset
train_test_dicts = {'train': train_dataset, 'val': val_dataset, 'test': test_dataset}

for d in ["train", "val", "test"]:
    DatasetCatalog.register("contest_" + d, lambda d=d: train_test_dicts[d])
    MetadataCatalog.get("contest_" + d).set(thing_classes=classes)
contest_metadata = MetadataCatalog.get("contest_train")

In [ ]:
# Verify registered dataset
dataset_dicts = test_dataset
for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=contest_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])
    print(d["file_name"])

## Model

In [11]:
# Data augmentation
def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [T.Resize((1024,1024)),
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                      T.RandomBrightness(0.6, 1.4),
                      T.RandomContrast(0.6, 1.4),
                      T.RandomSaturation(0.6, 1.4)
                      ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)

    return dataset_dict


# Custom trainer for COCO format dataset
class COCOTrainer(DefaultTrainer):

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"

        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [26]:
# Pretrained model
model_yaml = "COCO-InstanceSegmentation/mask_rcnn_R_101_DC5_3x.yaml"

# Customize model structure
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_yaml))
cfg.DATASETS.TRAIN = ("contest_train",)
cfg.DATASETS.TEST = ("contest_val",)
cfg.TEST.EVAL_PERIOD = 500
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_yaml)
cfg.SOLVER.IMS_PER_BATCH = 4                                # batch size
cfg.SOLVER.BASE_LR = 0.0001
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = (2000, 4000)                                  # decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512              # rpn proposal subset size for loss calculation
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.BACKBONE.FREEZE_AT = 0                            # freeze only stem
cfg.SOLVER.CHECKPOINT_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## Train

In [ ]:
trainer = COCOTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# Tensorboard training log
%load_ext tensorboard
%tensorboard --logdir output

## Inference


In [ ]:
weight_path = "model_weight.pth"
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, weight_path)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
for d in random.sample(test_dataset, 10):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=contest_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])
    print(d["file_name"])

In [ ]:
evaluator = COCOEvaluator("contest_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "contest_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))